<a href="https://colab.research.google.com/github/Abhijitgayen/Abhijitgayen/blob/main/LSTM%2BW2V%2BsemEval_2014_tweets.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

 ## Libary_enter

In [1]:
import tensorflow as tf
from tensorflow.keras.layers import Dense, LSTM
from tensorflow.keras.models import Model
from tensorflow.keras.models import Sequential
import numpy as np
import pandas as pd

import tensorflow_hub as hub
import numpy as np

In [2]:
# Load Pretrained Word2Vec
embed = hub.load("https://tfhub.dev/google/Wiki-words-250/2")

## Some_function for prepossing

In [3]:
def get_max_length(df):
    """
    get max token counts from train data, 
    so we use this number as fixed length input to RNN cell
    """
    max_length = 0
    for row in df['tweet']:
        if len(row.split(" ")) > max_length:
            max_length = len(row.split(" "))
    return max_length

def get_word2vec_enc(reviews):
    """
    get word2vec value for each word in sentence.
    concatenate word in numpy array, so we can use it as RNN input
    """
    encoded_reviews = []
    for review in reviews:
        tokens = review.split(" ")
        word2vec_embedding = embed(tokens)
        encoded_reviews.append(word2vec_embedding)
    return encoded_reviews
        
def get_padded_encoded_reviews(encoded_reviews):
    """
    for short sentences, we prepend zero padding so all input to RNN has same length
    """
    padded_reviews_encoding = []
    for enc_review in encoded_reviews:
        zero_padding_cnt = max_length - enc_review.shape[0]
        pad = np.zeros((1, 250))
        for i in range(zero_padding_cnt):
            enc_review = np.concatenate((pad, enc_review), axis=0)
        padded_reviews_encoding.append(enc_review)
    return padded_reviews_encoding

def sentiment_encode(sentiment):
    """
    return one hot encoding for Y value
    """
    if sentiment == 'positive':
        return [1,0]
    if sentiment == 'neutral':
        return [0.5,0.5]
    else:
        return [0,1]
    
def preprocess(df):
    """
    encode text value to numeric value
    """
    # encode words into word2vec
    reviews = df['tweet'].tolist()
    
    encoded_reviews = get_word2vec_enc(reviews)
    padded_encoded_reviews = get_padded_encoded_reviews(encoded_reviews)
    # encoded sentiment
    sentiments = df['label'].tolist()
    encoded_sentiment = [sentiment_encode(sentiment) for sentiment in sentiments]
    X = np.array(padded_encoded_reviews)
    Y = np.array(encoded_sentiment)
    return X, Y 

## Data_Set_add

In [4]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [11]:
tweetsDatas=pd.read_csv('/content/drive/MyDrive/2017_data_set/dataset/train/twitter-2015train-A.txt', sep="\t", header=None, names=["id","label", "tweet"]); #skiping these two rows as they have some bad data
testData=pd.read_csv('/content/drive/MyDrive/2017_data_set/dataset/train/twitter-2015test-A.txt',sep="\t", header=None, names=["id","label", "tweet"]); #test_data

In [12]:
print(tweetsDatas.head())
print(tweetsDatas.label.value_counts())
print(testData.head())
print(testData.label.value_counts())

                   id     label  \
0  105121481662541824  negative   
1  104319460143415296  negative   
2  103641916586999808   neutral   
3  100544750414864384   neutral   
4  102220476499243008   neutral   

                                               tweet  
0  I forgot how sad the first episode of the 5th ...  
1  Oct. 2 is almost too far away for me to wait f...  
2  (OFF TOPIC) - there is only 3 episodes on the ...  
3  @iheardavoicesay is finally catching up to me ...  
4  I downloaded the first season of #Dexter, gonn...  
neutral     253
positive    170
negative     66
Name: label, dtype: int64
                   id     label  \
0  520829332525441024  negative   
1  522931511323275264  positive   
2  523099837936312321  positive   
3  521384413217959937  positive   
4  523076584497229824  positive   

                                               tweet  
0  Saturday without Leeds United is like Sunday w...  
1  Catch Rainbow Valley at the @CBC #IMAF2014 Gal...  
2  @Nikla

In [14]:
df=tweetsDatas
# max_length is used for max sequence of input
max_length = get_max_length(df)
print(max_length)

train_X, train_Y = preprocess(df)

31


## ALL_models

In [7]:
# LSTM model
model = Sequential()
model.add(LSTM(128))
model.add(Dense(2, activation='softmax'))
model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

In [ ]:
# LSTM model
model = Sequential()
model.add(LSTM(128))
model.add(Dense(2, activation='softmax'))
model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

## train Models

In [15]:
print('Train...')
model.fit(train_X, train_Y,epochs=100)

Train...
Epoch 1/100
16/16 [==============================] - 1s 57ms/step - loss: 0.7460 - accuracy: 0.7342
Epoch 2/100
16/16 [==============================] - 1s 53ms/step - loss: 0.6645 - accuracy: 0.8057
Epoch 3/100
16/16 [==============================] - 1s 53ms/step - loss: 0.6460 - accuracy: 0.8446
Epoch 4/100
16/16 [==============================] - 1s 55ms/step - loss: 0.6606 - accuracy: 0.7812
Epoch 5/100
16/16 [==============================] - 1s 53ms/step - loss: 0.6364 - accuracy: 0.8037
Epoch 6/100
16/16 [==============================] - 1s 53ms/step - loss: 0.6296 - accuracy: 0.8200
Epoch 7/100
16/16 [==============================] - 1s 54ms/step - loss: 0.6191 - accuracy: 0.8098
Epoch 8/100
16/16 [==============================] - 1s 53ms/step - loss: 0.6106 - accuracy: 0.8200
Epoch 9/100
16/16 [==============================] - 1s 52ms/step - loss: 0.5989 - accuracy: 0.8282
Epoch 10/100
16/16 [==============================] - 1s 55ms/step - loss: 0.5860 - accurac

In [16]:
test_df=testData
print(test_df.label.value_counts())
max_length = get_max_length(test_df)
test_X, test_Y = preprocess(test_df)

score, acc = model.evaluate(test_X, test_Y, verbose=2)
print('Test score:', score)
print('Test accuracy:', acc)

positive    1038
neutral      987
negative     365
Name: label, dtype: int64
75/75 - 2s - loss: 1.3766 - accuracy: 0.6661 - 2s/epoch - 21ms/step
Test score: 1.3765665292739868
Test accuracy: 0.6661087870597839
